# 키워드 뉴스 검색 &  뉴스 크롤링 &  뉴스 요약

## gogamza/kobart-summarization 모델 사용

In [2]:
# 필요한 라이브러리 설치
!pip install transformers torch beautifulsoup4 requests sentence-transformers
!pip install sentencepiece feedparser googletrans==4.0.0-rc1

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from transformers import pipeline
import torch
import time
import random
from urllib.parse import urljoin, urlparse, quote
import warnings
import feedparser
import json
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

class WorkingNewsChatbot:
    def __init__(self):
        """뉴스 챗봇 초기화"""
        print(" 실제 작동하는 뉴스 요약 챗봇을 초기화하는 중...")

        # 한국어 요약 모델 로드
        print(" 한국어 요약 모델을 로드하는 중...")
        try:
            self.summarizer = pipeline(
                "summarization",
                model="gogamza/kobart-summarization",
                tokenizer="gogamza/kobart-summarization",
                device=0 if torch.cuda.is_available() else -1
            )
            print(" KoBART 요약 모델 로드 완료!")
        except Exception as e:
            print(f"  KoBART 모델 로드 실패, 대체 모델 사용: {e}")
            try:
                self.summarizer = pipeline(
                    "summarization",
                    model="facebook/bart-large-cnn",
                    device=0 if torch.cuda.is_available() else -1
                )
                print(" BART CNN 요약 모델 로드 완료!")
            except Exception as e2:
                print(f" 모든 요약 모델 로드 실패: {e2}")
                self.summarizer = None

        # 헤더 설정
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3',
        }

        # 실제 작동하는 RSS 피드 목록 (검증된 것들만)
        self.rss_feeds = {
            'bbc_korean': 'https://feeds.bbci.co.uk/korean/rss.xml',
            'yonhap': 'https://www.yna.co.kr/rss/news.xml',
            'hani': 'https://www.hani.co.kr/rss/',
            'khan': 'https://www.khan.co.kr/rss/rssdata/total_news.xml',
            'pressian': 'http://www.pressian.com/rss/news.xml',
            'ohmynews': 'http://rss.ohmynews.com/rss/news.xml'
        }

        print(" 챗봇 초기화 완료!\n")

    def search_google_news(self, keyword, max_articles=5):
        """Google News 검색 (개선된 버전)"""
        print(f" Google News에서 '{keyword}' 검색 중...")

        try:
            # Google News RSS 검색 (한국어)
            search_url = f"https://news.google.com/rss/search?q={quote(keyword)}&hl=ko&gl=KR&ceid=KR:ko"

            response = requests.get(search_url, headers=self.headers, timeout=15)
            response.raise_for_status()

            # RSS 파싱
            feed = feedparser.parse(response.content)
            articles = []

            print(f" Google News에서 {len(feed.entries)}개 항목 발견")

            for i, entry in enumerate(feed.entries[:max_articles]):
                try:
                    title = entry.title
                    link = entry.link
                    summary = entry.get('summary', '')
                    published = entry.get('published', '')
                    source = entry.get('source', {}).get('title', 'Google News')

                    # 기본 요약이 있으면 사용, 없으면 본문 크롤링 시도
                    content = summary
                    if len(content) < 100:  # 요약이 너무 짧으면 본문 크롤링
                        crawled_content = self.crawl_article_content(link)
                        if crawled_content:
                            content = crawled_content

                    if content and len(content) > 50:
                        articles.append({
                            'title': title,
                            'link': link,
                            'content': content,
                            'published': published,
                            'source': source
                        })
                        print(f" Google News 기사 {i+1}: {title[:50]}...")

                except Exception as e:
                    print(f"  Google News 기사 {i+1} 처리 오류: {e}")
                    continue

            return articles

        except Exception as e:
            print(f"  Google News 검색 오류: {e}")
            return []

    def search_working_rss_feeds(self, keyword, max_articles=5):
        """검증된 RSS 피드에서 뉴스 검색"""
        print(f" 검증된 RSS 피드에서 '{keyword}' 관련 뉴스 검색 중...")

        articles = []
        keyword_lower = keyword.lower()

        for source_name, rss_url in self.rss_feeds.items():
            if len(articles) >= max_articles:
                break

            try:
                print(f" {source_name.upper()} RSS 확인 중...")

                response = requests.get(rss_url, headers=self.headers, timeout=10)
                if response.status_code != 200:
                    print(f"  {source_name} RSS 접근 실패: {response.status_code}")
                    continue

                # RSS 파싱
                feed = feedparser.parse(response.content)

                if not feed.entries:
                    print(f"  {source_name} RSS에서 기사를 찾을 수 없음")
                    continue

                print(f" {source_name}에서 {len(feed.entries)}개 기사 확인 중...")

                found_count = 0
                for entry in feed.entries:
                    if len(articles) >= max_articles:
                        break

                    try:
                        title = entry.get('title', '')
                        link = entry.get('link', '')
                        summary = entry.get('description', entry.get('summary', ''))

                        # 키워드 매칭 확인 (제목 또는 요약에서)
                        if (keyword_lower in title.lower() or
                            keyword_lower in summary.lower()):

                            content = summary
                            if len(content) < 100:  # 요약이 짧으면 본문 크롤링 시도
                                crawled_content = self.crawl_article_content(link)
                                if crawled_content:
                                    content = crawled_content

                            if content and len(content) > 50:
                                articles.append({
                                    'title': title,
                                    'link': link,
                                    'content': content,
                                    'source': source_name.upper(),
                                    'published': entry.get('published', '')
                                })
                                found_count += 1
                                print(f" {source_name.upper()} 매칭 기사: {title[:50]}...")

                    except Exception as e:
                        print(f"  {source_name} 기사 처리 오류: {e}")
                        continue

                if found_count == 0:
                    print(f" {source_name}에서 '{keyword}' 관련 기사 없음")

                time.sleep(1)  # RSS 서버 부하 방지

            except Exception as e:
                print(f"  {source_name} RSS 오류: {e}")
                continue

        return articles

    def crawl_article_content(self, url):
        """기사 본문 크롤링 (단순화)"""
        try:
            if not url or 'google.com' in url:  # Google 리다이렉트 URL은 스킵
                return None

            response = requests.get(url, headers=self.headers, timeout=8)
            if response.status_code != 200:
                return None

            soup = BeautifulSoup(response.text, 'html.parser')

            # 일반적인 뉴스 본문 선택자들
            content_selectors = [
                'article',
                '.article-content',
                '.news-content',
                '.content',
                '.article-body',
                '[class*="content"]',
                '[class*="article"]'
            ]

            for selector in content_selectors:
                elements = soup.select(selector)
                if elements:
                    element = elements[0]
                    # 불필요한 태그 제거
                    for unwanted in element(["script", "style", "nav", "aside", "footer"]):
                        unwanted.decompose()

                    text = element.get_text(strip=True)
                    if len(text) > 200:
                        # 텍스트 정리
                        text = re.sub(r'\s+', ' ', text)
                        return text[:1500]  # 길이 제한

            return None

        except Exception:
            return None

    def summarize_text(self, text):
        """개선된 텍스트 요약"""
        try:
            if not self.summarizer:
                # 요약 모델이 없으면 첫 문장들만 반환
                sentences = text.split('.')[:2]
                return '.'.join(sentences).strip() + '.'

            if len(text) < 100:
                return text

            # 텍스트 전처리
            text = re.sub(r'\s+', ' ', text).strip()
            words = text.split()

            # 적절한 길이 설정
            text_length = len(words)
            if text_length < 50:
                return text

            # max_length를 입력 텍스트 길이에 맞게 조정
            max_len = min(150, text_length // 2)
            min_len = min(30, max_len - 10)

            if min_len >= max_len:
                min_len = max(10, max_len - 20)

            # 요약 실행
            summary = self.summarizer(
                text,
                max_length=max_len,
                min_length=min_len,
                do_sample=False,
                num_beams=3,
                length_penalty=1.0,
                early_stopping=True
            )

            result = summary[0]['summary_text']
            return result.strip()

        except Exception as e:
            print(f"  요약 중 오류: {e}")
            # 요약 실패 시 첫 두 문장만 반환
            sentences = text.split('.')[:2]
            return '.'.join(sentences).strip() + '.'

    def search_all_sources(self, keyword, max_articles=5):
        """모든 소스에서 뉴스 검색"""
        all_articles = []

        # 1. Google News 검색 (우선순위)
        print("=" * 50)
        google_articles = self.search_google_news(keyword, max_articles//2 + 1)
        all_articles.extend(google_articles)

        time.sleep(2)

        # 2. 검증된 RSS 피드 검색
        print("=" * 50)
        remaining = max_articles - len(all_articles)
        if remaining > 0:
            rss_articles = self.search_working_rss_feeds(keyword, remaining)
            all_articles.extend(rss_articles)

        return all_articles[:max_articles]

    def process_news_query(self, keyword):
        """뉴스 검색 및 요약 처리"""
        print("=" * 60)
        print(f" '{keyword}' 관련 뉴스 분석 결과")
        print("=" * 60)

        # 모든 소스에서 뉴스 검색
        articles = self.search_all_sources(keyword, max_articles=3)

        if not articles:
            return f" '{keyword}' 관련 뉴스를 찾을 수 없습니다.\n 다른 키워드를 시도해보세요. (예: AI, 경제, 정치, 스포츠)"

        print("=" * 50)
        print(" 기사 요약 중...")
        print("=" * 50)

        result = f" '{keyword}' 관련 최신 뉴스 ({len(articles)}건)\n\n"

        for i, article in enumerate(articles, 1):
            print(f"  기사 {i} 요약 처리 중...")

            # 기사 요약
            summary = self.summarize_text(article['content'])

            result += f" **기사 {i}**: {article['title']}\n"
            result += f" **출처**: {article.get('source', 'Unknown')}\n"
            result += f" **요약**: {summary}\n"
            result += f" **링크**: {article['link']}\n"
            if article.get('published'):
                result += f" **발행일**: {article['published']}\n"
            result += "-" * 50 + "\n\n"

        return result

    def start_chat(self):
        """챗봇 시작"""
        print(" 실제 작동하는 뉴스 요약 챗봇에 오신 것을 환영합니다!")
        print(" 궁금한 뉴스 키워드를 입력하면 여러 소스에서 최신 뉴스를 찾아 요약해드립니다.")
        print(" 지원 소스: Google News, 연합뉴스, BBC Korean, 한겨레, 경향신문")
        print(" 종료하려면 'quit', 'exit', '종료'를 입력하세요.")
        print(" 추천 키워드: AI, ChatGPT, 경제, 정치, 스포츠, 날씨, 주식, 북한\n")

        while True:
            try:
                user_input = input(" 검색할 뉴스 키워드를 입력하세요: ").strip()

                if not user_input:
                    print("  키워드를 입력해주세요.\n")
                    continue

                if user_input.lower() in ['quit', 'exit', '종료', 'q']:
                    print(" 뉴스 챗봇을 종료합니다. 안녕히 가세요!")
                    break

                # 뉴스 검색 및 요약
                result = self.#(user_input)
                print(result)

            except KeyboardInterrupt:
                print("\n 뉴스 챗봇을 종료합니다. 안녕히 가세요!")
                break
            except Exception as e:
                print(f" 오류가 발생했습니다: {e}\n")

# 챗봇 실행 함수들
def run_news_chatbot():
    """뉴스 챗봇 실행"""
    try:
        chatbot = WorkingNewsChatbot()
        chatbot.start_chat()
    except Exception as e:
        print(f" 챗봇 초기화 오류: {e}")

def test_single_query(keyword):
    """단일 키워드로 테스트"""
    try:
        chatbot = WorkingNewsChatbot()
        result = chatbot.process_news_query(keyword)
        print(result)
    except Exception as e:
        print(f" 테스트 오류: {e}")

# 실행 예시
if __name__ == "__main__":
    print(" 실제 작동하는 뉴스 요약 챗봇")
    print(" Google News + 검증된 한국 언론사 RSS 피드 활용")


    # 자동 실행 (주석 해제하여 사용)
    # run_news_chatbot()

    # 테스트 실행 (주석 해제하여 사용)
    # test_single_query('AI')

 실제 작동하는 뉴스 요약 챗봇
 Google News + 검증된 한국 언론사 RSS 피드 활용


In [11]:
test_single_query('손흥민')

 실제 작동하는 뉴스 요약 챗봇을 초기화하는 중...
 한국어 요약 모델을 로드하는 중...


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
Device set to use cuda:0


 KoBART 요약 모델 로드 완료!
 챗봇 초기화 완료!

 '손흥민' 관련 뉴스 분석 결과
 Google News에서 '손흥민' 검색 중...
 Google News에서 100개 항목 발견
 Google News 기사 1: LAFC 첫 선발서 도움 기록한 손흥민, MLS 베스트11 선정 - 뉴시스...
 Google News 기사 2: 손흥민 떠난 토트넘 구장에 태극기가?.. “광복절 알리고 싶었어요” - 조선일보...
 검증된 RSS 피드에서 '손흥민' 관련 뉴스 검색 중...
 BBC_KOREAN RSS 확인 중...
 bbc_korean에서 22개 기사 확인 중...
 bbc_korean에서 '손흥민' 관련 기사 없음
 YONHAP RSS 확인 중...
 yonhap에서 120개 기사 확인 중...
 YONHAP 매칭 기사: 유니폼 매진에 홈 데뷔전 티켓 최고가는 730만원…심상찮은 손흥민 효과...


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 기사 요약 중...
  기사 1 요약 처리 중...
  기사 2 요약 처리 중...
  기사 3 요약 처리 중...
 '손흥민' 관련 최신 뉴스 (3건)

 **기사 1**: LAFC 첫 선발서 도움 기록한 손흥민, MLS 베스트11 선정 - 뉴시스
 **출처**: 뉴시스
 **요약**: <a href="https://news.google.com/rss/articles/CBMiYEFVX3lxTE5mNVpyU09TcFVpM3l0V2Q5dC1iUmJTT3RxMDkwNmd6d0UtWlRRX09JVjJ3MHQySml3RUpWWUxVM2FEUEl6Y1BSZW1wTGlzVHktMEJwVVd2dllNU0F0Yjdlc9IBeEFVX3lxTE53UjlacWdkR3V3d2hBV0JQS2VXSUYzSlM0dl9VU0ZsaTRySUlaa01ZYkNhV1pSaXFzMEJFbGFIUDZJSllyVWdnN1RueFh6Q0FFeUhxZjlqZUpicWtPbTZHVjk3ZWxIXzlreWFoR3lsZGZ0Q084b1FTYg?oc=5" target="_blank">LAFC 첫 선발서 도움 기록한 손흥민, MLS 베스트11 선정</a>&nbsp;&nbsp;<font color="#6f6f6f">뉴시스</font>
 **링크**: https://news.google.com/rss/articles/CBMiYEFVX3lxTE5mNVpyU09TcFVpM3l0V2Q5dC1iUmJTT3RxMDkwNmd6d0UtWlRRX09JVjJ3MHQySml3RUpWWUxVM2FEUEl6Y1BSZW1wTGlzVHktMEJwVVd2dllNU0F0Yjdlc9IBeEFVX3lxTE53UjlacWdkR3V3d2hBV0JQS2VXSUYzSlM0dl9VU0ZsaTRySUlaa01ZYkNhV1pSaXFzMEJFbGFIUDZJSllyVWdnN1RueFh6Q0FFeUhxZjlqZUpicWtPbTZHVjk3ZWxIXzlreWFoR3lsZGZ0Q084b1FTYg?oc=5
 **발행일**: Tue, 19 Aug 2025 01:40:52 